<a href="https://colab.research.google.com/github/Sanidhya572/Hugging-Face-Transformers-/blob/main/chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r requirements.txt

In [2]:
from transformers import pipeline, Conversation
import gradio as gr

In [3]:
# toy example 1
pipeline(task="sentiment-analysis")("Love this!")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'POSITIVE', 'score': 0.9998745918273926}]

In [4]:
# toy example 2
pipeline(task="sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")("Love this!")

[{'label': 'POSITIVE', 'score': 0.9998745918273926}]

In [5]:
# defining classifier
classifier = pipeline(task="sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

In [6]:
classifier("Hate this.")

[{'label': 'NEGATIVE', 'score': 0.9997110962867737}]

In [7]:
# we can also pass in a list to classifier
text_list = ["This is great", \
             "Thanks for nothing", \
             "You've got to work on your face", \
             "You're beautiful, never change!"]

classifier(text_list)

[{'label': 'POSITIVE', 'score': 0.9998785257339478},
 {'label': 'POSITIVE', 'score': 0.9680057168006897},
 {'label': 'NEGATIVE', 'score': 0.8776116371154785},
 {'label': 'POSITIVE', 'score': 0.9998120665550232}]

In [8]:
# if there are multiple target labels, we can return them all
classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

In [9]:
classifier(text_list[0])

[[{'label': 'admiration', 'score': 0.9526104927062988},
  {'label': 'approval', 'score': 0.030472073704004288},
  {'label': 'neutral', 'score': 0.015236238949000835},
  {'label': 'excitement', 'score': 0.006063767243176699},
  {'label': 'gratitude', 'score': 0.005296194460242987},
  {'label': 'joy', 'score': 0.004475215449929237},
  {'label': 'curiosity', 'score': 0.004322330001741648},
  {'label': 'realization', 'score': 0.004089601803570986},
  {'label': 'optimism', 'score': 0.00407722033560276},
  {'label': 'disapproval', 'score': 0.004076560027897358},
  {'label': 'annoyance', 'score': 0.0035287411883473396},
  {'label': 'surprise', 'score': 0.0029730673413723707},
  {'label': 'disappointment', 'score': 0.002734638284891844},
  {'label': 'love', 'score': 0.00269457814283669},
  {'label': 'amusement', 'score': 0.0024867462925612926},
  {'label': 'confusion', 'score': 0.0023607409093528986},
  {'label': 'pride', 'score': 0.0021013382356613874},
  {'label': 'sadness', 'score': 0.00177

In [10]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [11]:
text = """
Hugging Face is an AI company that has become a major hub for open-source machine learning.
Their platform has 3 major elements which allow users to access and share machine learning resources.
First, is their rapidly growing repository of pre-trained open-source machine learning models for things such as natural language processing (NLP), computer vision, and more.
Second, is their library of datasets for training machine learning models for almost any task.
Third, and finally, is Spaces which is a collection of open-source ML apps.

The power of these resources is that they are community generated, which leverages all the benefits of open source i.e. cost-free, wide diversity of tools, high quality resources, and rapid pace of innovation.
While these make building powerful ML projects more accessible than before, there is another key element of the Hugging Face ecosystem—their Transformers library.
"""
summarized_text = summarizer(text, min_length=5, max_length=140)[0]['summary_text']
summarized_text

'Hugging Face is an AI company that has become a major hub for open-source machine learning. They have 3 major elements which allow users to access and share machine learning resources.'

In [12]:
classifier(summarized_text)

[[{'label': 'neutral', 'score': 0.9101784825325012},
  {'label': 'approval', 'score': 0.08781356364488602},
  {'label': 'realization', 'score': 0.02325625903904438},
  {'label': 'annoyance', 'score': 0.006623796187341213},
  {'label': 'admiration', 'score': 0.004981071222573519},
  {'label': 'disapproval', 'score': 0.0047301193699240685},
  {'label': 'optimism', 'score': 0.0033590728417038918},
  {'label': 'disappointment', 'score': 0.002619007369503379},
  {'label': 'confusion', 'score': 0.0019539804197847843},
  {'label': 'excitement', 'score': 0.001241705846041441},
  {'label': 'disgust', 'score': 0.0011407813290134072},
  {'label': 'joy', 'score': 0.0010540124494582415},
  {'label': 'amusement', 'score': 0.0009572388371452689},
  {'label': 'love', 'score': 0.0008871055324561894},
  {'label': 'desire', 'score': 0.0008553274674341083},
  {'label': 'curiosity', 'score': 0.000826106930617243},
  {'label': 'anger', 'score': 0.0007336385897360742},
  {'label': 'caring', 'score': 0.000697

In [13]:
chatbot = pipeline(model="facebook/blenderbot-400M-distill")

In [14]:
conversation = chatbot("Hi I'm Shaw, how are you?")

In [15]:
conversation

[{'generated_text': " I'm doing well. How are you doing this evening? I just got home from work."}]

In [16]:
conversation = chatbot("Where do you work?")

In [17]:
conversation

[{'generated_text': " I work at a grocery store as a cashier. It's not the most exciting job in the world, but it pays the bills."}]

In [18]:
def top3_text_classes(message, history):
    return str(classifier(message)[0][:3]).replace('}, {', '\n').replace('[{', '').replace('}]', '')

demo_sentiment = gr.ChatInterface(top3_text_classes, title="Text Sentiment Chatbot", description="Enter your text, and the chatbot will classify the sentiment.")

In [19]:
demo_sentiment.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [20]:
def summarizer_bot(message, history):
    return summarizer(message, min_length=5, max_length=140)[0]['summary_text']

demo_summarizer = gr.ChatInterface(summarizer_bot, title="Summarizer Chatbot", description="Enter your text, and the chatbot will return the summarized version.")

demo_summarizer.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [21]:
message_list = []
response_list = []

def vanilla_chatbot(message, history):
    conversation = chatbot(message)

    return conversation[0]['generated_text']

demo_chatbot = gr.ChatInterface(vanilla_chatbot, title="Vanilla Chatbot", description="Enter text to start chatting.")

demo_chatbot.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>